In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
listings.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,11204286,https://www.airbnb.com/rooms/11204286,20160706203047,2016-07-07,Family friendly/California king,"Aquatica Waterpark, Sleep train Amphitheater, ...","Walking to Aquatica Waterpark, Sleep train Amp...","Aquatica Waterpark, Sleep train Amphitheater, ...",none,NaN,...,10.0,f,NaN,"SAN DIEGO, SAN DIEGO TOURISM MARKETING DISTRIC...",f,moderate,f,f,2,4.57
1,7972006,https://www.airbnb.com/rooms/7972006,20160706203047,2016-07-07,Welcome to Sunset Suite,Your spacious room awaiting is with a Queen Si...,NaN,Your spacious room awaiting is with a Queen Si...,none,Getting around is easy. Very close to Eastlake...,...,10.0,f,NaN,NaN,f,strict,f,f,1,0.76
2,7727710,https://www.airbnb.com/rooms/7727710,20160706203047,2016-07-07,San Diego/Eastlake. Gated community,"This is an immaculate 3 bedroom, 2 1/2 bath co...",NaN,"This is an immaculate 3 bedroom, 2 1/2 bath co...",none,NaN,...,8.0,f,NaN,NaN,f,flexible,f,f,1,0.09
3,13124681,https://www.airbnb.com/rooms/13124681,20160706203047,2016-07-07,Townhome in Eastlake,This 2 Story TownHome is close to Otay Ranch ...,"My place is good for couples, business travele...",This 2 Story TownHome is close to Otay Ranch ...,none,"Located in eastern Chula Vista, Otay Ranch is ...",...,10.0,f,NaN,NaN,f,moderate,f,f,1,3.00
4,3469225,https://www.airbnb.com/rooms/3469225,20160706203047,2016-07-07,Bedroom suite in Large new home,Hello; we are offering a private secluded bedr...,"Beautiful, quiet award-winning suburban neighb...",Hello; we are offering a private secluded bedr...,none,"The quiet serenity; near Park and lakes, beaut...",...,NaN,f,NaN,NaN,f,strict,f,f,2,NaN


In [11]:
def street_zipcode_parse(x):
    st = ''
    
    if 'Mexico' in x:
            return '00000.0'
    
    for w in x:
        try:
            a = int(w)
        except ValueError:
            continue
        
        st = st + w
        
    return (st + '.0')

In [13]:
listings['street'].map(street_zipcode_parse).map(lambda x: x[-7:]).astype('float')

0       92154.0
1       91913.0
2       91914.0
3       91915.0
4       91915.0
5       91915.0
6       91915.0
7       91915.0
8       91915.0
9       91915.0
10      91915.0
11      91915.0
12      91915.0
13      91915.0
14      91910.0
15      91910.0
16      91910.0
17      91910.0
18      91910.0
19      91913.0
20      91915.0
21      91915.0
22      91913.0
23      91913.0
24      91915.0
25      91913.0
26      91913.0
27      91913.0
28      91913.0
29      91913.0
         ...   
6578    92014.0
6579    91941.0
6580    91950.0
6581    91950.0
6582    91902.0
6583    91915.0
6584    91942.0
6585    92110.0
6586    92109.0
6587    91950.0
6588    91913.0
6589        0.0
6590    92106.0
6591    92109.0
6592    92014.0
6593    91910.0
6594    92075.0
6595    92025.0
6596    92014.0
6597    92127.0
6598    91902.0
6599    91945.0
6600    92071.0
6601    92014.0
6602        0.0
6603    92014.0
6604    92014.0
6605    92075.0
6606    92075.0
6607    92075.0
Name: street, dtype: flo